# Residential Storage operation

This notebook explores two different operational strategies to improve a household's PV self-consumption with home battery storage systems: 
* The *greedy* strategy, which charges and discharges the storage whenever available and
* the *feed-in damp*, which aims to limit the power feed during peak PV generation.

A techno-economic analysis is performed with the help of [simses](https://gitlab.lrz.de/open-ees-ses/simses) a simulation framework for stationary energy storage systems.

In [1]:
import os
import pandas as pd

from simses.main import SimSES
from configparser import ConfigParser

In [2]:
pd.options.plotting.backend = "plotly"
# template = "plotly_dark"
template = "plotly_white"

# Greedy strategy

### Simulation parameters

In [3]:
os.path.abspath("../data")

'c:\\Users\\hessehoh\\Documents\\GIT\\23s_statbat\\data'

In [4]:
power    = 2000 # W # maximum power of the BESS System installed (here matched to Tesla Powerwall Type 2)
capacity = 13500 # Wh # Energy capacity of the BESS System installed (here matched to Tesla Powerwall Type 2)
loop_years = 1 # a - number of years to simulate via repetetive one-year profile execution

electricity_consumption = 10e6 # Yearly energy consumption in Wh (10e6 = 10 MWh)
pv_installed_capacity = 10e3   # PV peak-power in W --- 10e3 = 10 kWp

sim_params = f"""
[GENERAL]
START = 2014-01-01 00:00:00
END = 2014-12-31 23:59:59
TIME_STEP = 3600
LOOP = {loop_years}
EXPORT_DATA = True

[ENERGY_MANAGEMENT]
STRATEGY = ResidentialPvGreedy
MIN_SOC = 0.0
MAX_SOC = 1.0

[BATTERY]
START_SOC = 1.0
MIN_SOC = 0.0
MAX_SOC = 1.0
EOL = 0.6
START_SOH = 1

[STORAGE_SYSTEM]
; Configuration of the AC storage system:
; Format: AC-system name, max AC power in W, DC voltage level in V, ACDC converter name, housing name, HVAC name
STORAGE_SYSTEM_AC =
    system_1,{power},333,fix,no_housing,no_hvac

; Configuration of the AC/DC converter:
; Format: ACDC converter name, converter type, optional: number of converters
ACDC_CONVERTER =
    notton,NottonAcDcConverter
    fix,FixEfficiencyAcDcConverter,1,1,0.95

HOUSING =
    no_housing,NoHousing    
HVAC =
    no_hvac,NoHeatingVentilationAirConditioning

; Configuration of the DC storage system. Every AC system must have at least 1 DC system
; Format: AC-system name, DCDC converter name, storage technology name
STORAGE_SYSTEM_DC =
   system_1,no_loss,nmc

; Configuration of the DCDC converter
; Format: DCDC converter name, converter type, [efficiency]
DCDC_CONVERTER =
    no_loss,NoLossDcDcConverter

; Configuration of the storage technology.
; Format: storage technology name, energy in Wh, technology type, [technology specific parameters]
STORAGE_TECHNOLOGY =
    nmc,{capacity},lithium_ion,SanyoNMC
    lfp,{capacity},lithium_ion,SonyLFP
    ;LIB1,{capacity},lithium_ion,GenericCell

    
POWER_DISTRIBUTOR_AC = EqualPowerDistributor
POWER_DISTRIBUTOR_DC = EqualPowerDistributor
AMBIENT_TEMPERATURE_MODEL = ConstantAmbientTemperature,25
SOLAR_IRRADIATION_MODEL = NoSolarIrradiationModel
THERMAL_SIMULATION = False
CYCLE_DETECTOR = HalfCycleDetector


[PROFILE]
POWER_PROFILE_DIR = {os.path.abspath("../data")}
LOAD_PROFILE = simses_load_profile
GENERATION_PROFILE = simses_pv_profile
; LOAD_PROFILE_SCALING = The input load profile is scaled to energy or power or no scaling at all (False)
;   LOAD_SCALING_FACTOR = If LOAD_PROFILE_SCALING = Energy --> Annual energy in Wh
;   LOAD_SCALING_FACTOR = If LOAD_PROFILE_SCALING = Power --> Peak power in W

LOAD_PROFILE_SCALING = Energy
LOAD_SCALING_FACTOR = {electricity_consumption}
;LOAD_SCALING_FACTOR = 5e6

; GENERATION_PROFILE_SCALING = The generation profile is scaled to power or no scaling at all (False)
;   GENERATION_SCALING_FACTOR = If GENERATION_PROFILE_SCALING = Power --> Peak power in W
GENERATION_PROFILE_SCALING = Power
; GENERATION_SCALING_FACTOR = {pv_installed_capacity}
;GENERATION_SCALING_FACTOR = 5e3


"""
sim_config = ConfigParser()
sim_config.read_string(sim_params)

In [5]:
investment_costs = 5000  # €
electricity_price = 0.35 # €/kWh
pv_feedin_tariff = 0.08 # €/kWh
discount_rate = 0.00 # discounting to be considered as interest and debt is to be accounted for!

analysis_params = f"""
[ECONOMIC_ANALYSIS]
;Costs in Euro
INVESTMENT_COSTS = {investment_costs}
USE_SPECIFIC_COSTS = False

ELECTRICITY_PRICE = {electricity_price}
PV_FEED_IN_TARIFF = {pv_feedin_tariff}

DISCOUNT_RATE = {discount_rate}
"""
analysis_config = ConfigParser()
analysis_config.read_string(analysis_params)

In [6]:
path = os.path.abspath("..")
result_path = os.path.join(path, "simses_results").replace("\\", "/") + "/"

### Run simulation

In [7]:
simses = SimSES(path=result_path, name="greedy", simulation_config=sim_config, analysis_config=analysis_config)

In [8]:
simses.run()

[greedy: |####################| 100.0%]0%'}
          Duration in s: 17.94
Duration per step in ms: 0.98
['EES_20240409T155553M253171', 'EES_20240409T155810M614082', 'EES_20240409T155858M570214', 'EES_20240422T135820M809104', 'EES_20240422T140542M408944', 'EES_20240422T140848M388652', 'EES_20240422T141501M206037', 'EES_20240422T142156M974110', 'EES_20240424T082839M789905', 'EES_20250302T143053M899505', 'EES_20250302T143739M366624', 'EES_20250303T194240M552685', 'EES_20250310T173527M308625', 'EES_20250310T173609M665160']
c:/Users/hessehoh/Documents/GIT/23s_statbat/simses_results/greedy/EES_20250310T173609M665160/


### Load results

In [9]:
# results path
results_greedy = os.path.join(result_path, "greedy")
latest = os.listdir(results_greedy)[-1]
results_greedy = os.path.join(results_greedy, latest).replace("\\", "/")

In [10]:
df_greedy_sys = pd.read_csv(results_greedy + "/SystemState.csv.gz").drop_duplicates("Time in s")
df_greedy_ems = pd.read_csv(results_greedy + "/EnergyManagementState.csv.gz")
df_greedy_lis = pd.read_csv(results_greedy + "/LithiumIonState.csv.gz")

In [11]:
df_greedy_sys

,AC Fulfillment in p.u.,AC power in W,AC power delivered in W,Ambient temperature in K,Aux losses in W,Capacity in Wh,DC current in A,DC power additional in W,DC power of intermediate circuit in W,DC power loss in W,...,Outer layer temperature in K,PE losses in W,SOC in p.u.,State of Health in p.u.,Solar irradiation thermal load in W,DC power loss of storage technology in W,StorageSystemAC,StorageSystemDC,Internal air temperature in K,Time in s
0,1.0,0.000000,0.000000,298.15,0,13483.260000,0.000000,0,0.000000,0,...,0,0.000000,1.000000,1.000000,0.0,0.000000,1,0,298.15,1.388534e+09
2,1.0,-1400.425212,-1400.425212,298.15,0,13482.682396,-4.426822,0,-1474.131803,0,...,0,73.706590,0.890185,0.999957,0.0,6.529815,1,0,298.15,1.388538e+09
4,1.0,-1402.958191,-1402.958191,298.15,0,13482.336975,-4.434829,0,-1476.798096,0,...,0,73.839905,0.780140,0.999932,0.0,6.906657,1,0,298.15,1.388542e+09
6,1.0,-1063.013428,-1063.013428,298.15,0,13482.063358,-3.360245,0,-1118.961503,0,...,0,55.948075,0.696836,0.999911,0.0,4.170993,1,0,298.15,1.388545e+09
8,1.0,-602.574421,-602.574421,298.15,0,13481.825988,-1.904771,0,-634.288864,0,...,0,31.714443,0.649686,0.999894,0.0,1.395057,1,0,298.15,1.388549e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17510,-0.0,-1665.981556,0.000000,298.15,0,12182.572479,0.000000,0,0.000000,0,...,0,0.000000,0.000000,0.903533,0.0,0.000000,1,0,298.15,1.420052e+09
17512,-0.0,-1597.697514,0.000000,298.15,0,12182.563285,0.000000,0,0.000000,0,...,0,0.000000,0.000000,0.903532,0.0,0.000000,1,0,298.15,1.420056e+09
17514,-0.0,-1636.097374,0.000000,298.15,0,12182.554092,0.000000,0,0.000000,0,...,0,0.000000,0.000000,0.903532,0.0,0.000000,1,0,298.15,1.420060e+09
17516,-0.0,-1590.747112,0.000000,298.15,0,12182.544899,0.000000,0,0.000000,0,...,0,0.000000,0.000000,0.903531,0.0,0.000000,1,0,298.15,1.420063e+09


In [12]:
time = df_greedy_ems["Time in s"]
load = df_greedy_ems["Load in W"].values
pv_gen = df_greedy_ems["PV Generation in W"].values
residual_damp = load - pv_gen

power_greedy = df_greedy_sys["AC power delivered in W"].values
soc_greedy   = df_greedy_sys["SOC in p.u."].values

df_greedy = pd.DataFrame(
    data={
        "Residual in W": residual_damp,
        "Storage power in W": -power_greedy,
        "Storage SOC in p.u.": soc_greedy,
        "Grid power in W": residual_damp + power_greedy
    },
    index = pd.to_datetime(time, unit="s")
)

## Feed-in damp

### Parametrize and run simulation

In [26]:
df[["Residual in W", "Storage power in W (Greedy)"]].plot(template=template, labels={"value": "test"})

### State of Charge (SOC)

In [27]:
df[["Storage SOC in p.u. (Greedy)"]].plot(template=template, labels={"value": "SOC in p.u."})

### Grid power

In [29]:
df["Grid power in W (Greedy)"].plot(template=template, labels={"value": "Grid power in W"})

In [30]:
df[["Grid power in W (Greedy)"]].plot.hist(template=template, log_y=True, labels={"value": "Grid power in W"}).update_layout(barmode='overlay').update_traces(opacity=0.75)